In [ ]:
# basics
import utils
import numpy as np
from tqdm.notebook import tqdm 


# torch
import torch
from torch.utils.data import Dataset
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter



# custom modules
from data_acquisition import DataHandler
from data_preparation import apply_preprocessing_pipeline


# Configure logging for the pipeline
logger = utils.setup_logger(level='ERROR')

In [ ]:
cities = ['Aachen', 'CapeTown',]# 'Hamburg', 'Johannesburg', 'London', 'Montreal', 'Paris', 'Seoul', 'Singapore', 'Sydney']

datahandler = DataHandler(logger)


In [ ]:
# load images and mask for all specified cites

images = []
masks=[]
for city in tqdm(cities):
    images.append(datahandler.get_satellite_image(city))
    masks.append(datahandler.get_building_mask(city))

In [ ]:
# apply training pipeline
# TODO make train test split consistent so we can train with multiple sizes, dont know if there is an advantage though
train_loader, test_loader = apply_preprocessing_pipeline(images, masks, patch_size = 128, train_ratio = 0.8, batch_size = 64)

In [ ]:
# initialize model, taken from exercise pdf
model = nn.Sequential(
    nn.Conv2d(6, 32, kernel_size=3, padding=1), nn.ReLU(),
    nn.Conv2d(32, 64, kernel_size=3, padding=1), nn.ReLU(),
    nn.Conv2d(64, 128, kernel_size=3, padding=1), nn.ReLU(),
    nn.Conv2d(128, 1, kernel_size=1, padding=0),
    nn.Sigmoid())

# initialize tensorboard writer
writer = SummaryWriter()

In [ ]:
## ONLY FOR TESTING

# from data_preparation import *

# patched_images = create_tensor_of_windows(images[0], masks[0], patch_size=64)
# plot_sub_image(patched_images[300]).show()

In [ ]:
# Instantiate the model, loss function, and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
num_epochs = 50
for epoch in tqdm(range(num_epochs)):
    model.train()
    running_loss = 0.0
    for batch in train_loader:
        # splid in inputs and labels
        inputs = batch[:,:-1].to(torch.float32)
        labels = batch[:,-1, np.newaxis].to(torch.float32)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward pass
        outputs = model(inputs)

        # calculate loss
        loss = criterion(outputs, labels)

        # write to tensorboard
        writer.add_scalar("Loss/train", loss, epoch)

        # backward pass
        loss.backward()

        # optimizer step
        optimizer.step()
        running_loss += loss.item()
    


In [ ]:
writer.flush()

# Download

In [ ]:


buildings = []
sat_images = []
building_masks = []

for city in cities: 
    buildings.append(datahandler.get_buildings(city))
    sat_images.append(datahandler.get_satellite_image(city))
    building_masks.append(datahandler.get_building_mask(city))

# Plot the expected results for the first city 
datahandler.plot(city[0])

In [ ]:
import data_preparation

for city in cities:
    data_preparation.create_tensor(city)

# Download

In [ ]:
# Download 

for city in cities: 
    sat_image = datahandler.get_satellite_image(city)
    mask = datahandler.get_building_mask(city)

# Plot the expected results for the first city 
datahandler.plot(city[0])